In [1]:
import pyrosetta
import numpy as np
pyrosetta.init("-mute all -out:level 0")

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│              See LICENSE.md or email license@uw.edu for details              │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.Release.python310.linux 2024.08+release.717d2e8232174371f0c672564f23a097062db88a 2024-02-21T10:16:44] retrieved from: http://www.pyrosetta.org


In [2]:
def get_template_pose(seq,type="RNA",**kwargs):
    if type!="RNA":
        seq=seq.upper()
    else:
        seq=seq.lower()
    return pyrosetta.pose_from_sequence(seq,**kwargs)

In [3]:
def perturb_RNA(pose,seqlen=None,da=10,db=10,dg=10,dd=10,de=10,dz=10,dx=5,amplify=1.0):
    if seqlen is None:
        seqlen=len(sample_pose.secstruct())
    oldpose=pose.clone()
    rid=np.random.randint(seqlen)+1
    deltas=np.random.normal(np.zeros(7),np.array([da,db,dg,dd,de,dz,dx]).astype(float))*amplify
    pose.set_alpha(rid,pose.alpha(rid)+deltas[0])
    pose.set_beta(rid,pose.beta(rid)+deltas[1])
    pose.set_gamma(rid,pose.gamma(rid)+deltas[2])
    pose.set_delta(rid,pose.delta(rid)+deltas[3])
    pose.set_epsilon(rid,pose.epsilon(rid)+deltas[4])
    pose.set_zeta(rid,pose.zeta(rid)+deltas[5])
    pose.set_chi(rid,pose.chi(rid)+deltas[6])
    return oldpose,pose

In [5]:
class LoadedPDB(pyrosetta.rosetta.core.pose.Pose):
    def __init__(self,filename):
        pose=pyrosetta.pose_from_pdb(filename)
        super(LoadedPDB,self).__init__(pose)

    def __call__(self,chn=None,idx=None):
        if chn is None: return self
        else:
            if idx is None: return self.residue(int(chn))
            else: return self.residue(self.get_resid(chn,idx))
    def __len__(self): return self.pdb_info().nres()
    def all_phi(self): return np.array([self.phi(r+1) for r in range(len(self))])
    def all_psi(self): return np.array([self.psi(r+1) for r in range(len(self))])
    def all_chi(self): return np.array([self.chi(r+1) for r in range(len(self))])
    def generic_all(self,param):
        '''
        A generic function to get all "param"s (one from each residue).
        For example:
            generic_all("phi") -> Same as all_phi()
            generic_all("alpha") -> Returns a list of all alpha angles (RNA only)
        '''
        fx=getattr(self,param)
        ret=np.array([fx(r+1) for r in range(len(self))])
        return ret
    def all_generic(self,param): return self.generic_all(param)
    def get_resid(self,chn,idx): return self.pdb_info().pdb2pose(chn,int(idx))
    def get_atom(self,rid,aid):
        if type(rid)!=int: rid=rid.seqpos()
        if type(aid)==int: return pyrosetta.AtomID(aid,rid)
        else:
            residue=self.residue(rid)
            return pyrosetta.AtomID(residue.atom_index(aid),rid)
    def get_conformation_parameter(self,rid,par="bond_length",*args):
        conform=self.conformation()
        pass_param=[self.get_atom(rid,ai) for ai in args]
        return getattr(conform,par)(*pass_param)

In [4]:
seq="caacaacaacaa"
rna_seq = pyrosetta.rosetta.utility.vector1_std_string()
rna_seq.append(seq)
setup=pyrosetta.rosetta.core.import_pose.RNA_DeNovoSetup()
setup.set_sequence_strings(rna_seq)
setup.set_minimize_rna(True)
setup.set_silent_file("/home/venkata/dump.rta")

opts=setup.options()
opts.set_nstruct(10)

protocol = pyrosetta.rosetta.protocols.rna.denovo.RNA_DeNovoProtocol(opts, setup.rna_params())

In [8]:
startpose=get_template_pose(seq)
setup.set_native_pose(startpose)
#protocol.apply(startpose)